# Break it down
1. Decide what kind of data you want the application to manage. Examples include budget items, work request tickets, investment performance, book reviews, and more.
2. Create the .csv file with specific columns and a first row or two as "seed" data for your application to test while developing.
3. Create your index route and template that shows all items in the .csv file on the web page, including links to each individual item's details page.
4. Complete the show route that displays the details of each item in the .csv
5. Include a create route that allows users to add new data to the file. This should accept a POST request with the new data.


## Prelim

In [ ]:
!pip install pandas

In [ ]:
# Decide what kind of data you want the application to manage. 
# Examples include budget items, work request tickets, investment performance, 
# book reviews, and more.

import pandas as pd
data = pd.read_csv('./international_football_results.csv')
data.head()

In [ ]:
# Create the .csv file with specific columns and a 
# first row or two as "seed" data for your application to test while developing.

data['year'] = pd.to_datetime(data['date']).dt.year
data.head()

In [ ]:
data.sample(10).to_csv('seed.csv',index=False)

In [5]:
import pandas as pd
seed = pd.read_csv('seed.csv')
seed.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
0,2007-11-11,St. Martin,British Virgin Islands,0,1,Friendly,Marigot,St. Martin,False,2007
1,2005-10-08,Uganda,Burkina Faso,2,2,FIFA World Cup qualification,Kampala,Uganda,False,2005
2,1976-04-11,Kuwait,Saudi Arabia,3,1,Gulf Cup,Doha,Qatar,True,1976
3,2003-02-12,Croatia,Poland,0,0,Friendly,Split,Croatia,False,2003
4,1978-01-26,Sudan,Kenya,3,0,Friendly,Khartoum,Sudan,False,1978


In [6]:
seed['key'] = seed['home_team']+seed['away_team']+seed['date']
seed.to_csv('seed.csv',index=False)

## Flask

In [1]:
# Create your index route and template that shows all items in the 
# .csv file on the web page, including links to each individual 
# item's details page.

# setup
from flask import Flask, request, render_template
import csv

app = Flask(__name__) # invoke the Flask class

In [2]:
# step 1 - index route
@app.route('/index')
def index():
    data_list = []
    with open('seed.csv', mode='r') as file: 
        reader = csv.DictReader(file)
        for row in reader:
            data_list.append(row)
    # Pass the list of dictionaries to the template
    return render_template('index.html', data_list=data_list)

In [4]:
# run the server, test, and see! url is http://127.0.0.1:9001/
if __name__ == '__main__':
    app.run(port=9001) # Start the server listening for requests

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9001
Press CTRL+C to quit
127.0.0.1 - - [22/Apr/2024 19:06:55] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [22/Apr/2024 19:07:00] "GET /index HTTP/1.1" 200 -


```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Index</title>
</head>
<body>
    <h1>Soccer Games</h1>
    <ul>
        {% for data in data_list %}
        <li><a href="/show/{{data['key']}}">{{ data['tornament'] }}: {{ data['date'] }}</a></li>
        {% endfor %}
    </ul>

    <a href="/form">Add a new greeting!</a>
</body>
</html>
```
Example
```python
{'date': '2005-10-08',
  'home_team': 'Uganda',
  'away_team': 'Burkina Faso',
  'home_score': '2',
  'away_score': '2',
  'tournament': 'FIFA World Cup qualification',
  'city': 'Kampala',
  'country': 'Uganda',
  'neutral': 'False',
  'year': '2005'},
```